### Import Libraries

In [1]:
#Libs
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns

#preprocessing
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.feature_extraction.text import TfidfVectorizer

#Model Creation
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import GradientBoostingClassifier

#evaluation
#evaluation
from sklearn.metrics import confusion_matrix
from sklearn.metrics import ConfusionMatrixDisplay
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
from sklearn.metrics import f1_score


#hyperparameter tuning 
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV


### Import Data

In [2]:
#data
dim_cd_df = pd.read_csv('../data/interim_data/dim_cd_df.csv',index_col=0)
dim_qq_df = pd.read_csv('../data/interim_data/dim_qq_df.csv',index_col=0)
FACT_df = pd.read_csv('../data/interim_data/FACT_df.csv',index_col=0)

### Subset and Merge Data

In [3]:
#Subset the Fact table for only data from the US for 2020 and remove rows where the answer was NA
us_data = pd.merge(FACT_df,dim_cd_df[['Account_Num','Country','geometry']],on='Account_Num')

us_data.drop_duplicates(inplace=True)
us_data=us_data.query("Country == 'United States of America' & \
            Year ==2020 & \
            Answer not in ['Question not applicable','nan']")


In [4]:
#merge with dim_qq_df for the question section
us_data = pd.merge(us_data,dim_qq_df[['Question_ID','Sect']],on='Question_ID')

#drop null from answer
us_data.dropna(inplace=True)

In [5]:
#sample 5000 rows
us_data_sample = us_data.sample(5000).reset_index(drop=True)

us_data_sample.shape

(5000, 7)

In [6]:
us_data_sample.head()

,Year,Account_Num,Question_ID,Answer,Country,geometry,Sect
0,2020,10495,9039,Community-Scale Development > Green space and/...,United States of America,POINT (-115.14 36.1699),Mitigation Actions
1,2020,31108,1484,100000000,United States of America,POINT (-95.3694 29.7602),Adaptation Actions
2,2020,3417,16631,Local Government Operations Protocol (ICLEI/Th...,United States of America,POINT (-74.0059 40.7128),Local Government Operations GHG Emissions Data
3,2020,31177,12999,Yes,United States of America,POINT (-111.891 40.7608),Mitigation Target setting
4,2020,58621,9909,We are currently performing energy audits on m...,United States of America,POINT (-80.4139 37.2296),Mitigation Actions


In [22]:
#Analysis 7: Classification to predict geometry
us_data_sample_w_city =us_data_sample.merge(dim_cd_df[['Account_Num','City']],on='Account_Num').drop_duplicates().copy()
us_data_sample_w_city
X=us_data_sample_w_city[['Question_ID','Answer','Sect']]
y= us_data_sample_w_city['City']

In [23]:
#check for class imbalance
y.value_counts()

Houston             132
West Palm Beach     113
San Francisco       109
Washington, DC       95
Park City            94
                   ... 
Alton                 5
Roanoke               4
Toledo                4
Natchez               3
South Barrington      1
Name: City, Length: 116, dtype: int64

In [24]:
X

,Question_ID,Answer,Sect
0,9039,Community-Scale Development > Green space and/...,Mitigation Actions
3,9951,"Residential, commercial, resort, and governmen...",Mitigation Actions
6,19383,Short-term (by 2025),Water Supply
9,4622,Emissions occurring outside the city boundary ...,GCoM Emission Factor and Activity Data
12,3999,Integrated Elsewhere,City-wide GHG Emissions Data
...,...,...,...
14850,8307,Short-term (by 2025),Climate Hazards
14853,8466,Government capacity,Climate Hazards
14856,8729,Do not know,Climate Risk and Vulnerability Assessment
14859,16570,15000,City Details


In [25]:
X.dtypes

Question_ID     int64
Answer         object
Sect           object
dtype: object

In [26]:
X.Question_ID=X.Question_ID.astype('category')
X.Sect=X.Sect.astype('category')

/var/folders/r8/kdff6gm9553_wdcqb3x5r7vh0000gq/T/ipykernel_17204/3398063244.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X.Question_ID=X.Question_ID.astype('category')
/var/folders/r8/kdff6gm9553_wdcqb3x5r7vh0000gq/T/ipykernel_17204/3398063244.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X.Sect=X.Sect.astype('category')


In [27]:
# We will not set stratify=y and test moethods 1,3,4 and 5 listed above. 
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=.30, random_state=0)

#create unique lists for our different transformed columns
cat_cols = list(X_train.select_dtypes(include='category').columns)
text_cols = ('Answer')

#add the transformers
preprocessor = ColumnTransformer( 
    transformers = [
  
        ('cat',OneHotEncoder(handle_unknown = 'ignore'),cat_cols), #encode the categorical features
        ('Answer',TfidfVectorizer(stop_words='english',ngram_range=(1,2),lowercase=True),text_cols),    #freq counts for words
    ], remainder='passthrough')


#fit and transform our data
X_train_trans = preprocessor.fit_transform(X_train)
X_test_trans = preprocessor.transform(X_test)




#create our lm model
rf_model = RandomForestClassifier()
print("Basic RF Model")
rf_model.fit(X_train_trans,y_train)
rf_ypred=rf_model.predict(X_test_trans) 
f1_rf = f1_score(y_test,rf_ypred,average='weighted')
print(f1_rf)
print('----------------')

#create our knn model
param_grid = {'n_neighbors':np.arange(2,50)}

#model
knn_model = KNeighborsClassifier()
knn_cv_model= GridSearchCV(knn_model,param_grid,cv=5)

#fit
knn_cv_model.fit(X_train_trans,y_train)
knn_ypred = knn_cv_model.predict(X_test_trans)

f1_knn=f1_score(y_test,knn_ypred,average='weighted')
print("KNN Model W CV")
print(f1_knn)


Basic RF Model
0.09576652655955498
----------------


/Users/matt/opt/anaconda3/lib/python3.9/site-packages/sklearn/model_selection/_split.py:676: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  warnings.warn(
/Users/matt/opt/anaconda3/lib/python3.9/site-packages/sklearn/neighbors/_classification.py:228: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. Set `keepdims` to True or False to avoid this warning.
  mode, _ = stats.mode(_y[neigh_ind, k], axis=1)
/Users/matt/opt/anaconda3/lib/python3.9/site-packages/sklearn/neighbors/_classification.py:228: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts

/Users/matt/opt/anaconda3/lib/python3.9/site-packages/sklearn/neighbors/_classification.py:228: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. Set `keepdims` to True or False to avoid this warning.
  mode, _ = stats.mode(_y[neigh_ind, k], axis=1)
/Users/matt/opt/anaconda3/lib/python3.9/site-packages/sklearn/neighbors/_classification.py:228: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be 

/Users/matt/opt/anaconda3/lib/python3.9/site-packages/sklearn/neighbors/_classification.py:228: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. Set `keepdims` to True or False to avoid this warning.
  mode, _ = stats.mode(_y[neigh_ind, k], axis=1)
/Users/matt/opt/anaconda3/lib/python3.9/site-packages/sklearn/neighbors/_classification.py:228: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be 

/Users/matt/opt/anaconda3/lib/python3.9/site-packages/sklearn/neighbors/_classification.py:228: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. Set `keepdims` to True or False to avoid this warning.
  mode, _ = stats.mode(_y[neigh_ind, k], axis=1)
/Users/matt/opt/anaconda3/lib/python3.9/site-packages/sklearn/neighbors/_classification.py:228: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be 

...

/Users/matt/opt/anaconda3/lib/python3.9/site-packages/sklearn/neighbors/_classification.py:228: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. Set `keepdims` to True or False to avoid this warning.
  mode, _ = stats.mode(_y[neigh_ind, k], axis=1)
/Users/matt/opt/anaconda3/lib/python3.9/site-packages/sklearn/neighbors/_classification.py:228: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be 

/Users/matt/opt/anaconda3/lib/python3.9/site-packages/sklearn/neighbors/_classification.py:228: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. Set `keepdims` to True or False to avoid this warning.
  mode, _ = stats.mode(_y[neigh_ind, k], axis=1)
/Users/matt/opt/anaconda3/lib/python3.9/site-packages/sklearn/neighbors/_classification.py:228: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be 

/Users/matt/opt/anaconda3/lib/python3.9/site-packages/sklearn/neighbors/_classification.py:228: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. Set `keepdims` to True or False to avoid this warning.
  mode, _ = stats.mode(_y[neigh_ind, k], axis=1)
/Users/matt/opt/anaconda3/lib/python3.9/site-packages/sklearn/neighbors/_classification.py:228: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be 

KNN Model W CV
0.03092534551987546


/Users/matt/opt/anaconda3/lib/python3.9/site-packages/sklearn/neighbors/_classification.py:228: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. Set `keepdims` to True or False to avoid this warning.
  mode, _ = stats.mode(_y[neigh_ind, k], axis=1)


In [12]:
#confusion_matrix
cm= confusion_matrix(y_test, rf_ypred)
cm

array([[1, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [1, 0, 0, ..., 8, 0, 1],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

In [13]:
#classication report
print(classification_report(y_test,rf_ypred))


                  precision    recall  f1-score   support

        Abington       0.05      0.09      0.06        11
         Alameda       0.00      0.00      0.00         6
           Alton       0.00      0.00      0.00         1
       Anchorage       0.00      0.00      0.00        11
       Ann Arbor       0.23      0.24      0.23        21
       Arlington       0.00      0.00      0.00        14
       Asheville       0.00      0.00      0.00        18
           Aspen       0.31      0.22      0.26        18
          Aurora       0.00      0.00      0.00         2
          Austin       0.00      0.00      0.00        11
       Baltimore       0.00      0.00      0.00        20
      Blacksburg       0.00      0.00      0.00         7
     Bloomington       0.32      0.28      0.30        25
          Boston       0.19      0.18      0.18        28
         Boulder       0.25      0.26      0.26        23
  Boulder County       0.11      0.17      0.13         6
   Boynton Be

/Users/matt/opt/anaconda3/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/matt/opt/anaconda3/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/matt/opt/anaconda3/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
